In [2]:
conda activate DLC-GPU2


Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.interpolate import interp1d
import cv2
from tqdm import tqdm

from matplotlib.animation import FFMpegWriter
import matplotlib as mpl 
import wavio
mpl.rcParams['animation.ffmpeg_path'] = r'C:\Program Files\ffmpeg\bin\ffmpeg.exe'

In C:\Users\Niell lab\.conda\envs\DLC-GPU2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Niell lab\.conda\envs\DLC-GPU2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Niell lab\.conda\envs\DLC-GPU2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\Niell lab\.conda\envs\DLC-GPU2\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\Niell lab\.conda\env

In [6]:
cd T:\OptoPreyCapture\050721\PVCH13p3RT\trial_3

T:\OptoPreyCapture\050721\PVCH13p3RT\trial_3


In [7]:
top_file = '050721_PVCH13p3RT_laser_on_Rig2_trial_3_TOP1.nc'
top_data = xr.open_dataset(top_file)
top_vid_raw = np.uint8(top_data['TOP1_video'])
sz = top_vid_raw.shape
downsamp = 1
top_vid = np.zeros((sz[0],np.int(sz[1]*downsamp),np.int(sz[2]*downsamp)), dtype = 'uint8')
for f in range(sz[0]):
    top_vid[f,:,:] = cv2.resize(top_vid_raw[f,:,:],(np.int(sz[2]*downsamp),np.int(sz[1]*downsamp)))
topT = top_data.timestamps.copy()

In [8]:
# load data
imu_file = '050721_PVCH13p3RT_laser_on_Rig2_trial_3_imu.nc'
imu_data = xr.open_dataset(imu_file)
acc_chans = imu_data.IMU_data
ttl = np.array(acc_chans.sel(channel='acc_x'))
ttlT = imu_data.timestamps

# smooth and threshold
win = 10;
ttlSmooth = np.convolve(ttl,np.ones(win)/win,'same')
ttlOn = ttlSmooth>-1;
plt.plot(ttlT-ttlT[0],ttl, label = 'raw')
plt.plot(ttlT-ttlT[0],ttlSmooth, label = 'smoothed')
plt.plot(ttlT-ttlT[0],ttlOn, label = 'ttlOn')
plt.legend()

# interplolate
ttlInterp = interp1d(ttlT,ttlOn,'nearest', bounds_error = False)
ttlTop = ttlInterp(topT)  ### topT should be whatever you variable is for the timestamps of topdown camera



In [11]:
vidfile = 'T:\OptoPreyCapture\050721\PVCH13p3RT\trial_3\050721_PVCH13p3RT_laser_on_Rig2_trial_3_TOP1.avi'
writer = FFMpegWriter(fps=30)
max_frame = np.shape(top_vid)[0]
#max_frame = 300
fig =plt.figure(figsize = (6,6))

with writer.saving(fig, vidfile, 100):
    for f in tqdm(range(0,max_frame-1,2)):
        plt.cla()
        plt.imshow(top_vid[f,:,:],vmin=0,vmax = 255,cmap = 'gray')
        if ttlTop[f]==1:
            plt.plot(15,120,'b.',markersize=36)
        plt.axis('off')
        writer.grab_frame()
        
            

FileNotFoundError: [WinError 2] The system cannot find the file specified